In [1]:
import datetime
import logging
import os
import shutil
import time
from unittest.mock import Mock
import sys

import numpy as np
from io import StringIO
from pathlib import PurePath, Path

from ophyd import (set_cl, EpicsMotor, Signal, EpicsSignal,EpicsSignalWithRBV, EpicsSignalRO,
                   Component as Cpt)
from ophyd.utils.epics_pvs import (AlarmSeverity, AlarmStatus)                   

from ophyd.utils.paths import make_dir_tree
from ophyd import (SimDetector, SingleTrigger, Component, Device,
                   DynamicDeviceComponent, Kind, wait)
from ophyd.areadetector.plugins import (ImagePlugin, StatsPlugin,
                                        ColorConvPlugin, ProcessPlugin,
                                        OverlayPlugin, ROIPlugin,
                                        TransformPlugin, NetCDFPlugin,
                                        TIFFPlugin, JPEGPlugin, HDF5Plugin,
                                        register_plugin
                                        # FilePlugin
                                        )
from ophyd.areadetector.base import NDDerivedSignal
from ophyd.areadetector.filestore_mixins import (
    FileStoreTIFF, FileStoreIterativeWrite,
    FileStoreHDF5)

from ophyd.device import GenerateDatumInterface

# we do not have nexus installed on our test IOC
# from ophyd.areadetector.plugins import NexusPlugin
from ophyd.areadetector.plugins import PluginBase
from ophyd.areadetector.util import stub_templates
#from ophyd.device import (Component as Cpt, )
from ophyd.signal import Signal
import uuid

from epics import caget, caput, cainfo

from bluesky import RunEngine
from bluesky.utils import ProgressBarManager
from ophyd.sim import motor, det1, det 
from bluesky.simulators import summarize_plan
from bluesky.plans import count, scan
import bluesky.plan_stubs as bps
from bluesky.callbacks.best_effort import BestEffortCallback
import bluesky.preprocessors as bpp

RE = RunEngine({'scan_id': 0})

print(RE.md)
#RE.waiting_hook = ProgressBarManager()

from bluesky.log import config_bluesky_logging
config_bluesky_logging()
#config_bluesky_logging(level='DEBUG')

#bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
#RE.subscribe(bec)

from bluesky.callbacks.zmq import Publisher
publisher = Publisher('remote-dispatcher:5577')
RE.subscribe(publisher)


{'scan_id': 0, 'versions': {'ophyd': '1.6.2', 'bluesky': '1.6.7'}}


0

In [2]:
@register_plugin
class ZMQPlugin(PluginBase, GenerateDatumInterface): #, version_type='ADZMQ'):
    '''An areadetector plugin class that publishes an NDArray to a ZMQ socket'''
    _default_suffix = 'ZMQ1:'
    _suffix_re = r'ZMQ\d:'
    _plugin_type = 'NDPluginZMQ'
    
    _default_read_attrs = ['scan_id', 'unique_id']
    
    # ZMQ scan id
    scan_id = Cpt(EpicsSignal, 'ScanID', string=True)      


In [ ]:
cainfo('13SIM1:ZMQ1:ScanID')
cainfo('13SIM1:ZMQ1:ScanID_RBV')

In [3]:
class ADZMQStopAndShoot(SingleTrigger, SimDetector):
    zmq = Cpt(ZMQPlugin, 'ZMQ1:')
    md = None
    _default_read_attrs = ['zmq']
     
    def __init__(self, prefix='', *, name, kind=None, read_attrs=None,
                 configuration_attrs=None, parent=None, md=None, **kwargs):
        
        self.md = md
        
        print(f'*** Init, md = {self.md}')
        super().__init__(prefix=prefix, name=name, kind=kind, read_attrs=read_attrs, configuration_attrs=configuration_attrs, parent=parent, **kwargs)     
        
        print(f'*** zmq: {self.zmq.scan_id.get()}')
    
    def stage(self):
        print(f'*** stage, md = {self.md}')
        # Set zmq scan_id to stage_id in the passed meta-data dict
        self.zmq.stage_sigs['scan_id'] = str(self.md['scan_id'])
        
        print(f"**** pre stage, stage_id = {self.md['scan_id']}")

        super().stage()
        
        print(f'**** post stage, stage_id = {self.zmq.scan_id.get()}')
        
    def unstage(self):
        super().unstage()
        
        print(f'**** unstage, stage_id = {self.zmq.scan_id.get()}')
          
det_zmq = ADZMQStopAndShoot("13SIM1:", name='detector', md=RE.md)#, read_attrs='detector_zmq') #, md=RE.md)
det_zmq.cam.acquire_time.put(.01)
det_zmq.cam.num_images.put(1)
det_zmq.zmq.scan_id.put('')


*** Init, md = {'scan_id': 0, 'versions': {'ophyd': '1.6.2', 'bluesky': '1.6.7'}}
*** zmq: 0


In [6]:
def print_event(name, doc):
    print(f"Measured: {doc['data']}")
    
def print_start(name, doc):
    print(f"Start: uid = {doc['uid']}")  
        
RE(scan([det_zmq], motor, 0, 180, 180), {'event': print_event, 'start': print_start})

*** stage, md = {'scan_id': 2, 'versions': {'ophyd': '1.6.2', 'bluesky': '1.6.7'}}
**** pre stage, stage_id = 2
**** post stage, stage_id = 2
Start: uid = be5718c9-b16f-4bc4-ba1e-109e634940ff
Measured: {'detector_zmq_unique_id': 48, 'detector_zmq_scan_id': '2', 'motor': 0.0, 'motor_setpoint': 0.0}
Measured: {'detector_zmq_unique_id': 49, 'detector_zmq_scan_id': '2', 'motor': 1.005586592178771, 'motor_setpoint': 1.005586592178771}
Measured: {'detector_zmq_unique_id': 50, 'detector_zmq_scan_id': '2', 'motor': 2.011173184357542, 'motor_setpoint': 2.011173184357542}
Measured: {'detector_zmq_unique_id': 51, 'detector_zmq_scan_id': '2', 'motor': 3.016759776536313, 'motor_setpoint': 3.016759776536313}
Measured: {'detector_zmq_unique_id': 52, 'detector_zmq_scan_id': '2', 'motor': 4.022346368715084, 'motor_setpoint': 4.022346368715084}
Measured: {'detector_zmq_unique_id': 53, 'detector_zmq_scan_id': '2', 'motor': 5.027932960893855, 'motor_setpoint': 5.027932960893855}
Measured: {'detector_zmq_u

('be5718c9-b16f-4bc4-ba1e-109e634940ff',)